In [1]:
from finiteelementanalysis import pre_process as pre
from finiteelementanalysis import pre_process_demo_helper_fcns as pre_demo
from finiteelementanalysis import solver_demo_helper_functions as solver_demo
from finiteelementanalysis.solver import hyperelastic_solver
from finiteelementanalysis import visualize as viz
import numpy as np

def define_sample_problem_geom(ele_type, nx, ny, L, H):
    coords, connect = pre.generate_rect_mesh_2d(ele_type, 0.0, 0.0, L, H, nx, ny)
    return coords, connect

def define_sample_problem_info(ele_type, coords, connect, L, H):
    # Identify boundaries
    boundary_nodes, boundary_edges = pre.identify_rect_boundaries(coords, connect, ele_type, 0.0, L, 0.0, H)
    # 1. Fix left boundary: both u_x and u_y = 0.
    fixed_nodes = pre.assign_fixed_nodes_rect(boundary_nodes, "left", 0.0, 0.0)
    # Assign distributed load on the right boundary
    q = 10.0
    dload_info = pre.assign_uniform_load_rect(boundary_edges, "right", q, 0.0)
    # Assign material properties
    mu = 10
    kappa = 100
    material_props = np.array([mu, kappa])
    # Assign artificial displacement field
    displacement = np.zeros((coords.shape))
    for kk in range(0, coords.shape[0]):
        displacement[kk, 0] = coords[kk, 0] * 0.01
    return displacement, material_props, fixed_nodes, dload_info


In [2]:
ele_type = "D2_nn4_quad" # change this!
nx = 20 # change this to refine the mesh
ny = 15 # change this to refine the mesh
L = 10
H = 10
coords, connect = define_sample_problem_geom(ele_type, nx, ny, L, H)
displacement, material_props, fixed_nodes, dload_info = define_sample_problem_info(ele_type, coords, connect, L, H)

fname = "solver_mesh_1.png"
pre_demo.plot_mesh_2D(fname, ele_type, coords, connect)


In [3]:
# test how long assembley takes

num_runs = 5

avg_time_global_stiffness = solver_demo.time_assemble_global_stiffness(
        num_runs=num_runs,
        ele_type=ele_type,
        coords=coords.T,
        connect=connect.T,
        material_props=material_props,
        displacement=displacement.T
    )

avg_time_global_traction = solver_demo.time_assemble_global_traction(
        num_runs=num_runs,
        ele_type=ele_type,
        coords=coords.T,
        connect=connect.T,
        dload_info=dload_info
    )

avg_time_global_residual = solver_demo.time_assemble_global_residual(
        num_runs=num_runs,
        ele_type=ele_type,
        coords=coords.T,
        connect=connect.T,
        material_props=material_props,
        displacement=displacement.T
    )

print("avg time global stiffness:", avg_time_global_stiffness, "seconds")
print("avg time global traction:", avg_time_global_traction, "seconds")
print("avg time global residual:", avg_time_global_residual, "seconds")

avg time global stiffness: 0.34749155840836465 seconds
avg time global traction: 0.0013357833959162235 seconds
avg time global residual: 0.10123436660505832 seconds


In [4]:
# test how long the matrix solve takes

K, R = solver_demo.prep_for_matrix_solve(ele_type, coords.T, connect.T, material_props, displacement.T, fixed_nodes, dload_info)

method = "dense"
num_runs = 5
avg_time_dense_solve = solver_demo.time_one_matrix_solve(K, R, method, num_runs)

print("average time dense matrix solve:", avg_time_dense_solve, "seconds")
print("matrix shape: ", K.shape)


average time dense matrix solve: 0.008760199998505414 seconds
matrix shape:  (672, 672)


In [5]:
# look at the stiffness matrix to see sparse-ness

fname = "solver_global_stiffness_1.png"
solver_demo.analyze_and_visualize_matrix(K, fname)

In [6]:
# try a sparse matrix assembly option

num_runs = 5

avg_time_global_stiffness_sparse = solver_demo.time_assemble_global_stiffness_sparse(
        num_runs=num_runs,
        ele_type=ele_type,
        coords=coords.T,
        connect=connect.T,
        material_props=material_props,
        displacement=displacement.T
    )

print("avg time global stiffness assembly sparse:", avg_time_global_stiffness_sparse, "seconds")

avg time global stiffness assembly sparse: 0.11150377499870956 seconds


In [7]:
# try a sparse solver

method = "sparse"
num_runs = 10
avg_time_sparse_solve = solver_demo.time_one_matrix_solve(K, R, method, num_runs)

print("average time sparse matrix solve:", avg_time_sparse_solve)


average time sparse matrix solve: 0.0027883958071470262


In [8]:
# try another sparse solver

method = "sparse_iterative"
num_runs = 10
avg_time_sparse_iterative_solve = solver_demo.time_one_matrix_solve(K, R, method, num_runs)

print("average time sparse iterative matrix solve:", avg_time_sparse_iterative_solve)


average time sparse iterative matrix solve: 0.0009411084000021219


In [9]:
# run the example to look at the results

nr_num_steps = 5
nr_print = True

displacements_all, nr_info_all = hyperelastic_solver(material_props, ele_type, coords.T, connect.T, fixed_nodes, dload_info, nr_print, nr_num_steps, nr_tol=1e-9, nr_maxit=30)

fname = "disp.gif"
viz.make_deformation_gif(displacements_all, coords, connect, ele_type, fname)


Step 0, load factor = 0.200
Iteration 1, Correction=1.000000e+00, Residual=7.555330e-03, tolerance=1.000000e-09
Iteration 2, Correction=6.266198e-02, Residual=1.605372e-03, tolerance=1.000000e-09
Iteration 3, Correction=6.808903e-04, Residual=4.859855e-06, tolerance=1.000000e-09
Iteration 4, Correction=2.111449e-08, Residual=1.501351e-09, tolerance=1.000000e-09
Iteration 5, Correction=2.429620e-16, Residual=2.731306e-16, tolerance=1.000000e-09
Step 1, load factor = 0.400
Iteration 1, Correction=5.159850e-01, Residual=7.555330e-03, tolerance=1.000000e-09
Iteration 2, Correction=3.622430e-02, Residual=1.930811e-03, tolerance=1.000000e-09
Iteration 3, Correction=4.771034e-04, Residual=6.297713e-06, tolerance=1.000000e-09
Iteration 4, Correction=1.841046e-08, Residual=2.974476e-09, tolerance=1.000000e-09
Iteration 5, Correction=2.656909e-16, Residual=3.709279e-16, tolerance=1.000000e-09
Step 2, load factor = 0.600
Iteration 1, Correction=3.637297e-01, Residual=7.555330e-03, tolerance=1.000